<h1><center>Comparativo de Preços de Delivery de Supermercados da Região de Botucatu</center></h1>

Projeto realizado com o intuito de treinar técnicas de Web Scrapping. Esse código tem como inputs dois atributos, o produto e a marca, e retorna como output um dataframe com os resultados da busca nos dois principais supermercados da cidade de Botucatu.

In [1]:
import bs4
import urllib.request as urllib_request
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
def lista_produtos(produto,marca):
    marca = marca.upper()
    
    # Supermercado Central
    
    url_central = 'https://delivery.supercentral.com.br/loja/busca/?q='+produto+'&page=1'
    response_central = urlopen(url_central)
    html_central = response_central.read()
    soup_central = BeautifulSoup(html_central, 'html.parser')
    num_itens = int((soup_central.h1.get_text()).split()[0])
    if num_itens % 20 == 0:
        num_paginas = num_itens/20
    else:
        num_paginas = int(num_itens/20)+1

    lista_nome = []
    lista_preco = []

    for a in range(num_paginas):
        url_central = 'https://delivery.supercentral.com.br/loja/busca/?q='+produto+'&page='+str(a+1)
        response_central = urlopen(url_central)
        html_central = response_central.read()
        soup_central = BeautifulSoup(html_central, 'html.parser')
        for i in range(len(soup_central.findAll('span',{'class':'preco-final'}))):
            lista_nome.append(soup_central.findAll('a',{'class': 'link-nome-produto'})[i].get_text().replace('\n',''))
            lista_preco.append(soup_central.findAll('span',{'class':'preco-final'})[i].get_text().replace(',','.'))

    central = pd.DataFrame({'Supermercado':'Central','Produto':lista_nome , 'Preço':lista_preco})


    # Supermercado Jaú Serve

    url_jau = 'https://www.jauserve.com.br/on/demandware.store/Sites-JauServe-Site/pt_BR/Search-Show?q='+produto+'&lang=pt_BR&start=0&sz=100'
    response_jau = urlopen(url_jau)
    html_jau = response_jau.read()
    soup_jau = BeautifulSoup(html_jau, 'html.parser')

    lista_item = []
    lista_valor=[]
    
    for i in range(len(soup_jau.findAll('div',{'class':'pdp-link'}))):
        lista_item.append(soup_jau.findAll('div',{'class':'pdp-link'})[i].get_text().replace('\n','').upper())
        lista_valor.append(soup_jau.findAll('span',{'class':'sales mr-2'})[i].get_text().replace('\n','').split()[1].replace(',','.'))
        
    jau = pd.DataFrame({'Supermercado':'Jaú Serve','Produto':lista_item , 'Preço':lista_valor})


    
    # Combinação dos DataFrames

    dataset = central.merge(jau, how='outer')
    dataset['Preço'] = dataset['Preço'].astype(float)
    dataset = dataset.query(f'Produto.str.contains("{marca}")',engine='python')


 
    return dataset.sort_values('Preço').reset_index(drop=True)

In [6]:
lista_produtos('cerveja','brahma')

,Supermercado,Produto,Preço
0,Central,CERVEJA LATA 269 ML BRAHMA,2.59
1,Central,CERVEJA LATA 350 ML BRAHMA,3.09
2,Central,CERVEJA LATA 350 ML BRAHMA D. MALTE,3.29
3,Central,CERVEJA LATA 473 ML BRAHMA,3.99
4,Central,CERVEJA LATA 473 ML BRAHMA,3.99
5,Central,CERVEJA LATA 350 ML BRAHMA ZERO ÁLCOOL,3.99
6,Central,CERVEJA LONG NECK 330 ML BRAHMA DUPLO MALTE,4.19
7,Central,CERVEJA LATA 350 ML BRAHMA MALZBIER,4.29
8,Jaú Serve,CERVEJA BRAHMA ZERO LATA 350ML,4.29
9,Central,CERVEJA LONG NECK 355 ML BRAHMA ZERO,4.49
